In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def read_fasta(file):

    with open(file, "r") as f:
        lines = f.readlines()

    seq = ""
    seqs = []
    names = []

    for line in lines:
        if line.startswith(">"):
            names.append(line.strip().replace(">", ""))
            if len(names) > 1:
                seqs.append(seq.replace("*", ""))
                seq = ""
        else:
            seq += line.strip()

    seqs.append(seq.replace("*", ""))
    return pd.DataFrame({"name" : names, "aa_seq" : seqs})

In [34]:
# We want to calculate the probability that a sequence does contain FF --> Add probs of all sequences that do not contain FF

def no_FF_prob(l, F_prob):
    print(l)
    not_F_prob = 1 - F_prob
    if l == 2: 
        return F_prob * not_F_prob + not_F_prob * F_prob + not_F_prob * not_F_prob
    if l <= 1: 
        return 1
    else: 
        # Case where starts with F + case where doesn't start with F
        return F_prob * not_F_prob * no_FF_prob(l-2, F_prob) + not_F_prob * no_FF_prob(l-1, F_prob)

In [37]:
np.full(20, -1)

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1])

In [48]:
def no_FF_prob_dp(l, F_prob):
    not_F_prob = 1 - F_prob

    prob_ls = np.full(l, -1.0)

    # Base case: l = 1
    prob_ls[0] = 1
    prob_ls[1] = F_prob * not_F_prob + not_F_prob * prob_ls[0]

    for i in range(2, l):
        prob_ls[i] = F_prob * not_F_prob * prob_ls[i - 2] + not_F_prob * prob_ls[i-1]

    return prob_ls[-1]
    

In [8]:
cADs = read_fasta("../data/fasta_files/AllSeqs_IntegralAround_WxxLF_-50_+20_top138.fasta")
cADs["length"] = cADs['aa_seq'].str.len()
cADs["F_count"] = cADs['aa_seq'].str.count("F")
cADs["F_freq"] = cADs["F_count"] / cADs["length"]

In [61]:
F_prob

np.float64(0.07662251573830829)

In [56]:
F_prob = cADs["F_freq"].mean()
l = int(cADs["length"].mean())

In [57]:
1 - no_FF_prob_dp(l, F_prob)

np.float64(0.304491579905541)

In [58]:
1 - no_FF_prob_dp(50, F_prob)

np.float64(0.23637430431856066)

In [64]:
(1 - F_prob)**l

np.float64(0.004790841366379013)

In [62]:
F_prob

np.float64(0.07662251573830829)